# CIS530 Final Project - RumorEval Main Notebook
## Authors: Yoon Duk Kim, Peter Kong,  Sam Korn, Jason Tang, Noah Weiner

In [1]:
import sys
import os
import pickle
import pandas as pd
import numpy as np
import json

In [2]:
# import Pandas dataframe
from FileReader import FileReader
df_list = FileReader.get_dataframe()

saving data to output..


## Word Embeddings (if possible move code to separate file)

In [ ]:
# sys.path.insert(1, "./feature-extraction/embed-extractor")
# from EmbedExtractor import EmbedExtractor

# # Run this cell to generate the pre-PCA and post-PCA word embeddings
# print("oh")
# def chunkIt(seq, num):
#     print(len(seq))
#     avg = len(seq) / float(num)
#     out = []
#     last = 0.0
#     zeros = [0]*int(avg)

#     while last < len(seq):
#         subVector = seq[int(last):int(last + avg)]
#         if zeros==subVector and last>4700:
#             break
#         out.append(subVector)
#         last += avg
#     return out

####UNCOMMENT THIS TO RUN THE EE#####
# ee = EmbedExtractor()
# tweet2vec = {}
    
# for d in ["train","dev","test"]:
# 	with open('./output/full/'+d+'_data_full.json', 'r') as f:
# 	    jstr = f.read()

# 	j = json.loads(jstr)

# 	for key in j:
# 		tweet=j[key]['text']
# 		tweet2vec[key]=ee.tweetVec(tweet)
# pickleFile="feature-extraction/embed-extractor/word_embedding_vectors.pickle"
# pickle.dump(tweet2vec,open(pickleFile,"wb"))
# ##########################################


# from sklearn import decomposition
# from sklearn.preprocessing import StandardScaler
# word_embeddings_pca = {}
# pickleFile="feature-extraction/embed-extractor/word_embedding_vectors.pickle"
# word_embeddings = pd.read_pickle(pickleFile)

# word_embeddings_chunked = {}
# for word in word_embeddings:
#     word_embeddings_chunked[word]=chunkIt(word_embeddings[word],30)
# for word in word_embeddings_chunked:
#     pca = decomposition.PCA(n_components=24)
#     x = np.array(word_embeddings_chunked[word])
#     print(len(x))
#     try:
#         x_std = StandardScaler().fit_transform(x)
#         pca.fit_transform(x_std)
#         word_embeddings_pca[word]=pca.singular_values_
# #         print(len(pca.singular_values_))
#     except ValueError:
#         # print(word_embeddings_chunked[word])
#         print("UH OH")

# pickle.dump(word_embeddings_pca, open("feature-extraction/embed-extractor/word_embedding_vectors_pca.pickle","wb"))
# for word in word_embeddings_pca:
#     if len(word_embeddings_pca[word])!=16:
#         print("OH!")

## Opinion (If possible move code to separate file or elsewhere in code)

In [4]:
def create_opinion_column(df, strongly_subj_list):
    #add a binary column where opinion == 1 if the tweet text contains a strongly subjective word
    #global strongly_subj_list
    OpinionExtractor.add_opinion_column(df, strongly_subj_list)

## User Features (if possible move code to separate file)

In [5]:
from datetime import datetime

def extract_user_column(row):
    global user_labels
        
    user_dict = row['user']
    
    for col in user_dict.keys():
        user_labels.append(col)

def update_user_column(row):
    global user_vals
    user_dict = row['user']
    
    for key in user_vals.keys():
        concat_key = key[5:]
        if concat_key in user_dict:
            val = user_dict[concat_key]
            user_vals[key].append(val)
        else:
            user_vals[key].append(np.nan)
            
def convert_date(row):
    date = row['user_created_at'].split()
    date_str = ' '.join([date[1], date[2], date[-1]])
    
    datetime_object = datetime.strptime(date_str, '%b %d %Y')
    date_int = datetime_object.year * 10000 + datetime_object.month * 100 + datetime_object.day
        
    return date_int

def convert_to_int(row, col):
    return int(row[col])

def normalize_column(df, col):
    col_array = np.asarray(df[col].tolist())
    mean = np.mean(col_array)
    std = np.std(col_array)
    col_array = (col_array - mean) / float(std)
    
    df[col] = col_array
    
def create_user_features(df):

    global user_labels
    global user_vals
    user_labels = []
    df.apply(extract_user_column, axis = 1)
    user_labels = ['user_' + label for label in set(user_labels)]
    user_vals = {label:[] for label in user_labels}
    
    df.apply(update_user_column, axis = 1) 
    
    user_df = pd.DataFrame(user_vals)
    user_df['user_created'] = user_df.apply(convert_date, axis = 1)

    col_list = ['user_default_profile', 
                'user_favourites_count', 'user_followers_count', 'user_friends_count', 'user_geo_enabled',
                'user_listed_count', 'user_statuses_count', 'user_verified','user_created']

    for col in col_list:
        user_df[col] = user_df.apply(lambda x : convert_to_int(x, col), axis = 1)

    # normalize_column(user_df, col)
    user_df = user_df[col_list]    

    norm_list = ['user_favourites_count', 'user_followers_count', 'user_friends_count',
                'user_listed_count', 'user_statuses_count','user_created']

    for col in norm_list:
        normalize_column(user_df, col)

    df = pd.concat([df.reset_index(), user_df], axis = 1)
    df = df.set_index('index')

    return df   

## More embed stuff... commented out for now. Move to separate file if possible

In [6]:
# with open('feature-extraction/embed-extractor/word_embedding_vectors.pickle', 'rb') as pickle_file:
#     ee = pickle.load(pickle_file)

# for e in ee.keys():
#     print("key = " + str(e) + "   " + str(len(ee[e])))

# ee = EmbedExtractor()

## Build labels and vectorizations

In [13]:
sys.path.insert(1, "./feature-extraction/vulgar-extractor")
from VulgarExtractor import VulgarExtractor
sys.path.insert(1, "./feature-extraction/twitter-parser")
from TwitterParser import TwitterParser
sys.path.insert(1, "./feature-extraction/opinion-extractor/")
from OpinionExtractor import OpinionExtractor

def labels_and_vectors(file, index=0):
    # load dataframe
    df = pd.read_pickle(file)
    

    # Get POS tags
    textlist = [txt.replace('\n','') for txt in df['text'].tolist()]
    tagged_sents = TwitterParser.tag(textlist)
    df['POS'] = tagged_sents


    # Preprocess text (maybe eliminate)
    processed_sents = []
    for tagged_sent in df['POS']:
        processed_words = []
        for word, tag in tagged_sent:
            if tag == 'U':
                processed_words.append('someurl')
            elif tag == '@':
                processed_words.append('@someuser')
            else:
                processed_words.append(word)
        sent = ' '.join(processed_words)
        processed_sents.append(sent)
    df['text'] = processed_sents


    # vulgar extractor
    wordlist = VulgarExtractor.vulgarWords("feature-extraction/vulgar-extractor/badwords.txt") 
    dftext = df[['text']]
    result = dftext.applymap(lambda x: VulgarExtractor.containsVulgar(x,wordlist))
    df['isVulgar'] = result


    # word embeddings
    # word_embeddings = [ee[key] for key in df.index]
    # word_embeddings = [ee.tweetVec(tagged_line) for tagged_line in df['text']]
    # df['wordEmbedding'] = word_embeddings


    # Twitter text features
    word_counts = [TwitterParser.word_count(tagged_line) for tagged_line in df['POS']]
    contains_adjs = [TwitterParser.contains_adjectives(tagged_line) for tagged_line in df['POS']]
    contains_urls = [TwitterParser.contains_url(tagged_line) for tagged_line in df['POS']]
    contains_emojis = [TwitterParser.contains_emoji(tagged_line) for tagged_line in df['POS']]
    contains_abbrevs = [TwitterParser.contains_abbreviation(tagged_line) for tagged_line in df['POS']]

    df['wordCount'] = word_counts
    df['containsAdjective'] = contains_adjs
    df['containsURL'] = contains_urls
    df['containsEmoji'] = contains_emojis
    df['containsAbbreviation'] = contains_abbrevs


    # opinion feature
    strongly_subj_list = OpinionExtractor.initialize_subjectivity()
    create_opinion_column(df, strongly_subj_list)
    

    # user features
    df = create_user_features(df)    
      

    # attributes
    attributes = [
        # 'isVulgar',
        # 'containsAdjective',
        'containsURL',
        # 'containsEmoji',
        # 'containsAbbreviation',
        'wordCount',
        # 'opinion',
        # 'wordEmbeddings'
    ]
        
    attributes.extend([
        'num_replies',
        # 're_has_?',
        # 're_has_NOT',
        're_has_correct',
        're_has_credib',
        # 're_has_data',
        # 're_has_detail',
        # 're_has_fabricat',
        # 're_has_lie',
        're_has_proof',
        're_has_source',
        're_has_witness',
        # 'user_default_profile',
        # 'user_favourites_count',
        # 'user_followers_count',
        # 'user_friends_count',
        # 'user_geo_enabled',
        # 'user_listed_count',
        # 'user_statuses_count',
        # 'user_verified',
        # 'user_created'
    ])

    # labels
    # Changes "true"/"false"/"unverified" to numeric values, just like the in the early cells
    df.loc[df.classification == 'true', 'classification'] = 1
    df.loc[df.classification == 'false', 'classification'] = 0
    df.loc[df.classification == 'unverified', 'classification'] = 2
        
    labels = df['classification']
    labels = [l for l in labels]
    labels = np.array(labels)


    # getting the values as a list of lists
    values = df[attributes].values.tolist()


    # UNCOMMENT THIS IN ORDER TO INCOPORATE WORD_EMBEDDINGS AGAIN
    # word_embedding_values = df['wordEmbedding'].values.tolist()
    # for i,d in enumerate(word_embedding_values):
    #     values[i].extend(d)

    values = np.array(values)
    if index == 1:
        return df.index, values
    else:
        print(attributes)
    
    return labels, values

In [14]:
tr_labels, tr_values = labels_and_vectors('output/full/train_data_full.pickle')
indices, dev_values = labels_and_vectors('output/full/goldtest_data_full.pickle', index=1)

['containsURL', 'wordCount', 'num_replies', 're_has_correct', 're_has_credib', 're_has_proof', 're_has_source', 're_has_witness']


## Classifier Setup

In [15]:
import Classifiers

predictions, probabilities = Classifiers.naive_bayes(tr_values, tr_labels, dev_values)
# predictions, probabilities = classifiers.random_forest(tr_values, tr_labels, dev_values, 80, 3, "gini")
# predictions, probabilities = classifiers.decision_tree_classifier(tr_values, tr_labels, dev_values")
# predictions, probabilities = classifiers.svm_classifier(tr_values, tr_labels, dev_values")

In [16]:
# convert the numerical predictions back into their string values
ps = []
for i, p in enumerate(predictions):
    if p == 0:
        ps.append('false')
    if p == 1:
        ps.append('true')
    if p == 2:
        ps.append('unverified')

In [17]:
# creates pairings of the prediction and the probability of the prediction
pred_probs_pairs = [[ps[i], probabilities[i][predictions[i]]] for i in range(len(predictions))] 

# attaches the tweetID (called reference_id in the score.py file)
pred_dict = {index:pred_probs_pairs[i] for i,index in enumerate(indices)}

In [18]:
# writes the output
output_dir = './output/'
try:
    os.stat(output_dir)
except:
    os.mkdir(output_dir)  

with open('output/test_predictions.json', 'w') as outfile:
    json.dump(pred_dict, outfile)